<a href="https://colab.research.google.com/github/SahilSuvarna1023/T5-BART-Summarizer/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install rouge_score
!pip install evaluate
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, BartForConditionalGeneration, BartTokenizer
import evaluate
from evaluate import load

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and

In [3]:
def clean_dataset(df):
    # Drop missing values
    df = df.dropna()
    # Remove duplicates
    df = df.drop_duplicates()
    # Trim whitespace
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    return df

In [4]:
def load_dataset(file_path, sample_size=50):
    df = pd.read_csv(file_path)
    print("Columns in dataset:", df.columns.tolist())  # Debugging line

    # Ensure correct column name
    if 'article' in df.columns:
        df = df.rename(columns={'article': 'text'})
    else:
        raise KeyError("Expected column 'article' not found in dataset. Please check column names.")

    df = clean_dataset(df)
    df = df.sample(n=sample_size, random_state=42)  # Select 50 random samples
    return df

In [5]:
def summarize_text_t5(text, model, tokenizer, max_length=150):
    input_text = "summarize: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def summarize_text_bart(text, model, tokenizer, max_length=150):
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def evaluate_summaries(references, predictions):
    rouge = load("rouge")
    results = rouge.compute(predictions=predictions, references=references)
    return results


In [ ]:
def main():
    # Load dataset (only 50 samples)
    dataset_path = "/content/test.csv"  # Update with correct column names
    df = load_dataset(dataset_path, sample_size=50)

    # Load models
    t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")
    t5_tokenizer = T5Tokenizer.from_pretrained("t5-small", legacy=False)
    bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
    bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

    # Summarize and evaluate
    df['t5_summary'] = df['text'].apply(lambda x: summarize_text_t5(x, t5_model, t5_tokenizer))
    df['bart_summary'] = df['text'].apply(lambda x: summarize_text_bart(x, bart_model, bart_tokenizer))

    # Evaluate with Rouge
    rouge_scores = evaluate_summaries(df['text'].tolist(), df['t5_summary'].tolist())
    print("ROUGE Scores:", rouge_scores)

    # Save results
    df.to_csv("/content/summarized_results.csv", index=False)
    print("Summarization complete. Results saved.")

if __name__ == "__main__":
    main()

Columns in dataset: ['id', 'article', 'highlights']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]